In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.6 MB/s eta 0:00:00


In [ ]:
!pip install gensim


In [ ]:
import pandas as pd
import nltk
import gensim
from fuzzywuzzy import fuzz
#from transformers import BertTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Initialize NLTK's SnowballStemmer for stemming
stemmer = nltk.stem.SnowballStemmer('english')
# Load the CSV data into a DataFrame
df = pd.read_csv('formulation.csv')
df1=df
df['SYMPTOMs']=df['Symptoms']


# Thesaurus or synonym dictionary to expand symptoms
synonym_dict = {
   "Fever": ["high temperature", "pyrexia", "febrile"],
    "Cough": ["coughing", "persistent cough", "throat irritation"],
    "Inflammation": ["swelling", "redness", "inflammatory response"],
    "Hepatitis": ["liver inflammation", "jaundice", "hepatic disorder"],
    "Anemia": ["iron deficiency", "low red blood cell count", "blood disorder"],
    "Uremia": ["uremic poisoning", "high levels of urea in the blood"],
     "Jaundice": ["yellowing of skin", "icterus", "bilirubin excess"],
    "Bronchial Fever": ["fever with bronchial involvement"],
    "Asthma": ["breathing difficulty", "respiratory distress", "wheezing"],
    "Running-Nose": ["runny nose", "nasal congestion", "rhinorrhea"],
    "Body-ache": ["body pain", "muscle soreness", "aches all over"],
    "Non-healing-wounds": ["slow-healing wounds", "chronic wounds", "wounds that won't heal"],
    "ulcers": ["sores", "open sores", "ulceration"],
    "skin-tuberculosis": ["cutaneous tuberculosis", "TB of the skin"],
    "ringworm-infection": ["tinea", "dermatophytosis", "fungal skin infection"],
    "Piles": ["hemorrhoids", "haemorrhoids", "piles of the rectum"],
   "hemorrhoids": ["piles", "hemroids"],
    "bloating": ["abdominal bloating", "stomach bloating", "abdominal distension"],
    "constipation": ["irregular bowel movement", "difficulty passing stools", "bowel irregularity"],
    "digestion": ["digestive system function", "metabolism"],
    "difficulty-in-urination": ["urinary retention", "painful urination", "urinary difficulties"],
    "anorexia": ["loss of appetite", "appetite loss", "decreased appetite"],
    "Chronic fever": ["persistent fever", "long-term fever", "prolonged fever"],
    "malaria": ["malarial fever"],
    "indigestion": ["dyspepsia", "upset stomach", "impaired digestion"],
    "immunity": ["immune system strength", "resistance"],
    "Dermatitis": ["skin inflammation", "skin irritation"],
    "Eczema": ["atopic dermatitis", "skin rash"],
    "acne": ["pimples", "zits", "skin blemishes"],
    "hives": ["urticaria", "skin rash with raised, itchy bumps"],
    "rosacea": ["facial redness", "skin flushing"],
    "vitiligo": ["skin depigmentation", "loss of skin color"],
    "itching": ["pruritus", "skin irritation"],
    "dry cracked-skin": ["dry and cracked skin", "dehydrated skin"],
    "redness": ["skin redness", "skin irritation"],
    "painful-lesions": ["painful sores", "sore spots"],
    "pale-skin": ["pallor", "lack of skin color"],
    "Growth": ["growth development", "physiological growth"],
    "Bone-strength": ["skeletal strength", "bone health"],
    "appetite": ["hunger", "desire to eat"],
    "angina pectoris": ["chest pain", "heart-related chest pain"],
   "high blood pressure": ["hypertension", "elevated blood pressure", "high BP"],
    "myocardial infarction": ["heart attack", "coronary infarction"],
    "congestive heart failure": ["heart failure", "cardiac failure"],
    "isochemic cardiomyopathy": ["ischemic cardiomyopathy", "heart muscle disease due to reduced blood supply"],
    "cardiac arrhythmia": ["irregular heartbeat", "heart rhythm disorder"],
    "Mitral Regurgitation": ["MR", "leaky heart valve"],
    "Menorrhagia": ["heavy menstrual bleeding", "excessive menstrual bleeding"],
    "Metrorrhagia": ["abnormal uterine bleeding between periods"],
    "Menometrorrhagia": ["prolonged or heavy menstrual bleeding"],
    "dysmenorrhea": ["painful periods", "menstrual cramps"],
    "Menstrual cramps": ["period pain", "dysmenorrhea", "menstrual discomfort", "menstrual pain"],
    "hormonal problems": ["hormonal imbalances", "endocrine issues"],
    "polyuria": ["frequent urination", "excessive urination"],
    "PCOS": ["polycystic ovary syndrome"],
    "menstrual disorder": ["irregular periods", "menstrual irregularities"],
    "heavy menstrual bleeding": ["menorrhagia", "excessive menstrual bleeding"],
    "Stress": ["mental stress", "emotional stress", "psychological stress"],
    "Anxiety": ["nervousness", "worry", "apprehension", "fear", "anxiousness"],
    "Epilepsy": ["seizure disorder", "convulsive disorder"],
    "fatigue": ["tiredness", "exhaustion", "weariness"],
    "male sexual dysfunction": ["erectile dysfunction", "impotence", "sexual performance issues"],
    "arthritis": ["joint inflammation", "joint pain", "arthritic condition"],
    "sleeplessness": ["insomnia", "difficulty falling asleep", "sleep problems"],
    "insomnia": ["sleeplessness", "lack of sleep", "sleep disorder"],
    "facial paralysis": ["Bell's palsy", "loss of facial movement"],
    "muscle cramps": ["muscle spasms", "cramping", "muscle contractions"],
    "physical weakness": ["weakness", "lack of strength", "feebleness"],
    "bronchitis": ["respiratory infection", "inflammation of bronchial tubes"],
    "splitting pain": ["sharp pain", "severe pain", "intense pain"],
    "Pyuria": ["pus in urine", "presence of pus cells in urine"],
    "Kidney Stone": ["renal calculi", "nephrolithiasis", "stones in the kidney"],
    "kidney failure": ["renal failure", "kidney dysfunction", "end-stage renal disease"],
    "gonorrhea": ["clap", "drip", "venereal disease", "VD"],
    "Burning sensation during urination": ["painful urination", "dysuria", "burning while urinating"],
    "Frequent urination": ["urinary frequency", "urinating often", "increased urination"],
    "Frequent urge to urinate but incomplete voiding": ["urge to pee often", "feeling of incomplete emptying of the bladder"],
    "Pain or pressure in the back or lower abdomen": ["back pain", "lower abdominal pain", "abdominal pressure"],
    "Pelvic pain": ["pain in pelvis", "lower abdominal pain"],
    "Blood in urine": ["hematuria", "bloody urine", "urine with blood"],
    "Loss of appetite": ["decreased appetite", "lack of hunger"],
    "postpartum fever": ["fever after childbirth", "fever postpartum", "post-birth fever"],
    "postpartum diarrhea": ["diarrhea after childbirth", "post-birth diarrhea"],
    "low backache after delivery": ["back pain after childbirth", "post-birth backache"],
    "improves immunity": ["enhances immunity", "boosts immune system"],
    "recurrent miscarriage": ["repeated miscarriage", "multiple miscarriages"],
    "pus in seminal fluid": ["pus in semen", "seminal fluid with pus"],
    "osteoporosis": ["brittle bones", "porous bones"],
    "weak bones": ["fragile bones", "brittle bones"],
    "pelvic inflammation": ["inflammation in the pelvic region", "pelvic inflammatory disease"],
    "uterus sluggishness": ["sluggish uterus", "lazy uterus"],
    "gas": ["flatulence", "intestinal gas"],
    "flatulence": ["excessive gas", "bloated feeling"],
    "headache": ["head pain", "cephalalgia"],
    "gallbladder disorder": ["gallbladder issue", "gallbladder problem"],
    "burping": ["belching", "eructation"],
    "tiredness": ["fatigue", "exhaustion"],
    "restlessness": ["agitation", "nervousness"],
    "postpartum debility": ["weakness after childbirth", "post-birth fatigue"],
    "diarrhea": ["loose motions", "frequent bowel movements"],
    "low milk supply": ["insufficient breast milk", "inadequate lactation"],
    "Chest injury": ["chest trauma", "thoracic injury"],
    "tuberculosis": ["TB", "consumption"],
    "Liver problems": ["hepatic issues", "hepatic disorders"],
    "liver enlargement": ["hepatomegaly", "enlarged liver"],
    "pain in upper abdominal region": ["upper abdominal pain", "epigastric pain"],
    "fullness": ["abdominal fullness", "feeling full"],
    "dysuria": ["painful urination", "discomfort during urination"],
    "pain during urination": ["pain while urinating", "discomfort during urination"],
    "cramps before menstrual period": ["premenstrual cramps", "menstrual cramps"],
    "cramping": ["abdominal cramping", "muscle cramping"],
    "abdominal pain": ["stomach ache", "belly pain"],
    "pimple": ["zit", "acne"],
    "intestinal worms": ["worm infestation", "helminthiasis"],
    "leprosy": ["Hansen's disease"],
    "psoriasis": ["skin disorder characterized by red patches", "chronic skin condition"],
    "skin allergies": ["allergic skin reactions", "dermatitis"],
    "swollen lymph nodes": ["enlarged lymph nodes", "lymphadenopathy"],
    "leakage of fluid from skin": ["weeping skin", "oozing from the skin"],
    "crust formation": ["skin crusts", "crusted skin"],
    "scaly skin": ["flaky skin", "scaling of the skin"],
    "Excessive bleeding (Menorrhagia)": ["heavy menstrual bleeding", "excessive menstrual flow"],
    "Irregular periods (Menstrual irregularities)": ["menstrual irregularities", "irregular menstrual cycles"],
    "Leucorrhea": ["vaginal discharge", "whitish discharge"],
    "Uterine infections": ["infections of the uterus", "uterine infections"],
    "Hemorrhoids (Piles)": ["piles", "hemorrhoidal disease"],
    "Skin disorders": ["skin conditions", "dermatological issues"],
    "anaemia": ["anemia", "low red blood cell count"],
    "iron deficiency": ["low iron levels", "deficiency of iron"],
    "weakness": ["fatigue", "lack of strength"],
     "splenomegaly": ["enlarged spleen", "spleen enlargement"],
    "anaemic dropsy": ["dropsy due to anemia", "anemic dropsy"],
    "spleen enlargement": ["enlargement of the spleen", "spleen swelling"],
    "dysentery": ["bloody diarrhea", "intestinal inflammation"],
    "gastroenteritis": ["stomach flu", "gastrointestinal infection"],
    "poor digestion": ["digestive issues", "digestive problems"],
    "poor metabolism": ["metabolic issues", "metabolic problems"],
    "digestive disorder": ["gastrointestinal disorder", "digestive system disorder"],
    "malabsorption": ["impaired nutrient absorption", "absorption issues"],
    "dyspepsia": ["indigestion", "upset stomach"],
    "Bronchitis": ["respiratory tract inflammation", "chest infection"],
    "irritable bowel disease": ["IBD", "inflammatory bowel disease"],
    "abdominal distension": ["bloating", "swollen abdomen"],
    "Spleen disorders": ["spleen issues", "problems with spleen"],
    "Memory problems": ["memory issues", "memory difficulties"],
    "Nervous system disorders": ["neurological disorders", "nerve system issues"],
    "Bleeding Disorders": ["hemorrhagic disorders", "coagulation disorders"],
    "Urinary Disorders": ["urinary system disorders", "urinary tract issues"],
     "foul smelly urine": ["malodorous urine", "bad-smelling urine", "strong-smelling urine"],
    "Respiratory congestion": ["breathing congestion", "congested respiration"],
    "Wheezing": ["whistling sound while breathing", "breathing with a wheeze"],
    "Allergies": ["allergic reactions", "hypersensitivities"],
    "worm infestation": ["intestinal worm infection", "parasitic infestation"],
    "splenic disease": ["disease of the spleen", "issues with the spleen"],
    "foul smelly urine": ["malodorous urine", "bad-smelling urine", "strong-smelling urine"],
    "Respiratory congestion": ["breathing congestion", "congested respiration"],
    "Wheezing": ["whistling sound while breathing", "breathing with a wheeze"],
    "Allergies": ["allergic reactions", "hypersensitivities"],
    "worm infestation": ["intestinal worm infection", "parasitic infestation"],
    "splenic disease": ["disease of the spleen", "issues with the spleen"],
    "Digestive disorders": ["gastrointestinal disorders", "digestive system problems"],
    "Stomach Gas and bloating": ["abdominal bloating", "excessive gas in the stomach"],
    "Respiratory issues": ["breathing problems", "respiratory challenges"],
    "Stomach Pain": ["abdominal pain", "gastric pain"],
    "uterine smooth muscle(during periods)": ["uterine muscle contractions during menstruation", "smooth muscle contractions during periods"],
     "dryness of stool": ["hard stools", "dehydration in stools"],
    "irregular appetite": ["unpredictable hunger", "fluctuating appetite"],
    "abdominal gurgling and anxiety": ["stomach rumbling and anxiousness", "nervous stomach sounds"],
    "bleeding": ["hemorrhage", "oozing blood"],
    "respiratory disorder": ["breathing problem", "lung condition"],
    "Common cold": ["cold", "upper respiratory infection"],
    "Allergic rhinitis": ["hay fever", "allergy-related runny nose"],
    "Hiccup": ["hiccough", "singultus"],
    "chronic sinus": ["long-lasting sinus", "persistent sinus"],
    "sinusitis": ["sinus infection", "sinus inflammation"],
    "nausea": ["queasiness", "feeling sick"],
    "vomiting": ["throwing up", "emesis"],
    "poor digestion": ["indigestion", "digestive issues"],
    "Inability to concentrate": ["difficulty concentrating", "focus problems"],
   "Low retention power": ["poor memory retention", "memory problems"],
    "Memory Loss": ["amnesia", "forgetfulness", "memory problems"],
    "Mental fatigue": ["brain fatigue", "cognitive exhaustion"],
    "Depression": ["major depressive disorder", "clinical depression"],
    "Insomnia (sleeplessness)": ["sleep problems", "lack of sleep", "sleeplessness"],
    "Behavior disorders": ["behavioral disorders", "conduct disorders"],
    "Amnesia": ["memory loss", "forgetfulness", "memory problems"],
    "Alzheimer Disease": ["Alzheimer's", "AD", "dementia"],
    "Migraine": ["migraine headache", "severe headache", "migraine attack"],
    "Trigeminal Neuralgia": ["TN", "tic douloureux", "facial pain"],
    "Flatulence and bloating": ["excessive gas", "bloating", "abdominal distension"],
    "Weak digestion and metabolism": ["poor digestion", "slow metabolism", "digestive issues"],
     "Lack of appetite": ["loss of appetite", "decreased appetite", "poor appetite"],
    "Immune system support": ["boosting immune system", "strengthening immunity", "immune health"],
    "Respiratory health": ["lung health", "breathing health", "respiratory well-being"],
    "General weakness and fatigue": ["fatigue", "tiredness", "lack of energy"],
    "Heartburn": ["acid reflux", "gastroesophageal reflux", "pyrosis"],
    "Dizziness": ["lightheadedness", "vertigo", "unsteadiness"],
    "Small and raised skin bumps": ["papules", "skin elevations", "skin lumps"],
    "Leaking fluid from the skin": ["skin discharge", "oozing from the skin", "skin weeping"],
    "Bad or foul odor from the skin due to any underlying disease": ["malodorous skin", "foul-smelling skin", "unpleasant skin odor"],
    "Swollen skin": ["skin swelling", "skin edema", "skin puffiness"],
    "allergy": ["allergic reaction", "hypersensitivity", "allergic response"],
    "Gastrointestinal infections": ["intestinal infections", "GI infections", "gut infections"],
    "Loose stools": ["diarrhea", "watery stools", "runny stools"],
    "skin diseases": ["dermatological conditions", "skin disorders", "cutaneous diseases"],
    "ascites": ["fluid retention in the abdomen", "abdominal fluid accumulation", "abdominal swelling due to fluid"],
    "fistula": ["abnormal passage", "anomalous connection", "unnatural tunnel"],
    "COPD": ["chronic obstructive pulmonary disease", "chronic lung disease", "respiratory condition"],
    "Chronic Bronchitis": ["persistent bronchitis", "long-term bronchial inflammation", "prolonged bronchial infection"],
    "nosebleed": ["epistaxis", "bleeding from the nose", "nasal hemorrhage"],
    "dry mouth": ["xerostomia", "decreased saliva production", "mouth dryness"],
    "excessive thirst": ["polydipsia", "abnormal thirst", "extreme thirst"],
    "vertigo": ["dizziness", "spinning sensation", "loss of balance"],
    "Leucorrhoea": ["vaginal discharge", "whitish discharge", "vaginal fluid"],
    "General Weakness": ["overall weakness", "lack of energy", "fatigue"],
    "infertility": ["reproductive issues", "sterility", "inability to conceive"],
    "erectile dysfunction": ["impotence", "sexual impotence", "ED", "difficulty achieving or maintaining an erection"],
    "loss of energy": ["fatigue", "lack of energy", "tiredness", "exhaustion"],
    "persistent weakness": ["continuous weakness", "ongoing lack of strength", "constant fatigue"],
    "weakness after childbirth": ["postpartum weakness", "fatigue following childbirth", "weakness post-delivery"],
    "low milk production": ["insufficient milk supply", "inadequate lactation", "not enough breast milk"],
     "inflammation in reproductive organs": ["reproductive organ inflammation", "inflammation in genitals", "genital inflammation"],
    "lack of vitality": ["vitality deficiency", "energy loss", "decreased vitality", "lacking in vitality"],
    "abdominal cramps": ["stomach cramps", "belly cramps", "abdominal pain", "cramping in the abdomen"],
    "menstrual pain": ["period pain", "dysmenorrhea", "menstrual cramps", "pain during menstruation"],
    "irregular menstrual cycles": ["unpredictable menstrual cycles", "menstrual irregularities", "irregular periods"],
    "pain in abdomen": ["abdominal pain", "stomach pain", "belly ache", "abdominal discomfort"],
    "cardiac pain": ["heart pain", "chest pain", "pain in the chest", "cardiac discomfort"],
    "cardiac discomfort": ["heart discomfort", "chest discomfort", "discomfort in the chest", "cardiac unease"],
    "Chronic inflammatory lung disease": ["chronic lung inflammation", "persistent lung inflammation", "chronic lung disorder", "inflammatory lung condition"],
    "rhinitis": ["runny nose", "nasal inflammation", "inflammation of the nose", "nasal congestion"],
    "loss of voice": ["hoarseness", "voice loss", "no voice", "vocal cord issues"],
     "aphasia": ["speech disorder", "language impairment", "difficulty speaking", "communication disorder"],
    "jwara": ["fever", "pyrexia", "high body temperature", "elevated temperature"],
    "flu": ["influenza", "grippe", "viral flu", "flu-like illness"],
    "dry coughs": ["non-productive cough", "tickly cough", "irritative cough", "unproductive cough"],
    "persistent cough": ["chronic cough", "long-lasting cough", "lingering cough", "continual cough"],
    "hoarseness (hoarse voice)": ["rough voice", "husky voice", "scratchy voice", "hoarseness"],
    "dyspnea due to lung diseases or respiratory disorders": ["breathing difficulty due to lung issues", "shortness of breath from respiratory conditions", "labored breathing due to lung disorders", "respiratory distress"],
    "mucus": ["phlegm", "sputum", "respiratory discharge", "lung secretion"],
    "cataract and bronchitis": ["cataracts and bronchial inflammation", "eye lens opacity and bronchial inflammation"],
    "treatment of indigestion": ["indigestion management", "indigestion treatment", "alleviation of indigestion"],
    "Loss of interest to eat": ["decreased appetite", "lack of desire to eat", "reduced appetite"],
    "Burning sensation": ["burning feeling", "burning perception", "sensation of burning"],
    "Excessive thirst": ["unquenchable thirst", "abnormal thirst", "constant desire to drink"],
     "Nausea and vomiting with sour taste of mouth": ["nausea and sour taste while vomiting", "sour-tasting vomit with nausea", "vomiting with sour mouth taste"],
    "sour burp": ["acidic burping", "sour belch", "sour eructation"],
    "Premature greying of hair": ["early hair graying", "premature gray hair", "premature hair discoloration"],
    "hairfall": ["hair loss", "shedding hair", "falling hair"],
    "difficult breathing": ["labored breathing", "breathing challenges", "breathing difficulty"],
    "cataract": ["clouding of the eye lens", "opacification of eye lens", "eye lens cloudiness"],
    "frozen shoulder": ["adhesive capsulitis", "restricted shoulder movement", "stiff shoulder"],
    "weak eyesight": ["poor eyesight", "vision weakness", "low vision"],
    "pain in ankle": ["ankle ache", "ankle discomfort", "aching ankle"],
    "pain in flanks": ["side pain", "flank ache", "discomfort in the side"],
    "pain in joints and pelvis region": ["joint and pelvis pain", "discomfort in joints and pelvis", "aching joints and pelvis"],
    "rheumatic complaints": ["rheumatic issues", "rheumatological problems", "complaints related to rheumatism"],
    "Irritability": ["easily irritated", "prone to irritation", "irritable behavior"],
    "Sweating": ["perspiration", "sweatiness", "excessive sweating"],
    "Body aches": ["generalized body pain", "aching all over", "body discomfort"],
    "muscle aches": ["aching muscles", "sore muscles", "muscle discomfort"],
    "Dehydration": ["lack of hydration", "excessive fluid loss", "thirst"],
    "burning urination": ["painful urination", "urinary burning", "stinging urination"],
    "cleanses urinary bladder": ["urinary bladder cleansing", "bladder detoxification", "purification of the bladder"],
    "kidney disorder": ["renal disorder", "issues with kidneys", "kidney problems"],
   "vomiting burning sensation": ["burning sensation during vomiting", "vomiting with burning feeling"],
    "heartburn": ["acid indigestion", "burning chest pain", "acid reflux"],
    "gout": ["gouty arthritis", "joint inflammation due to excess uric acid", "uric acid-induced joint pain"],
    "wounds": ["injuries", "cuts", "abrasions"],
    "rheumatoid arthritis": ["RA", "inflammatory joint disease", "autoimmune arthritis"],
    "reduced digestive power": ["weakened digestion", "impaired digestive capacity", "digestive weakness"],
    "constipation.": ["difficulty in bowel movement", "hard stools", "bowel movement problem"],
    "Urolithiasis": ["urinary tract stones", "kidney stones", "urinary calculi"],
    "gouty arthritis": ["gout", "joint inflammation due to excess uric acid", "uric acid-induced joint pain"],
    "swelling in joints": ["joint swelling", "edema in joints", "inflammation in joints"],
    "redness in joints": ["joint redness", "inflammation causing redness in joints"],
    "renal calculi": ["kidney stones", "urinary tract stones", "nephrolithiasis"],
    "tumors": ["neoplasms", "abnormal tissue growth", "cancerous growths"],
    "cystic swelling": ["swelling with cysts", "fluid-filled swelling", "cystic growths"],
    "hypothyroidism": ["underactive thyroid", "low thyroid function", "thyroid hormone deficiency"],
    "hyperthyroidism": ["overactive thyroid", "excessive thyroid hormone", "thyrotoxicosis"],
    "non-cancerous breast tumor": ["benign breast tumor", "non-malignant breast lump", "fibroadenoma"],
    "endometriosis": ["growth outside the uterus", "endometrial tissue outside the uterus", "endometrial disorder"],
    "Increased uric acid": ["hyperuricemia", "high levels of uric acid in blood", "elevated uric acid"],
    "boils": ["furuncles", "skin abscesses", "painful lumps with pus"],
    "bursitis": ["inflammation of the bursa", "bursal inflammation", "swelling near joints"],
    "sprain": ["ligament injury", "stretched or torn ligament", "ligament strain"],
    "inflammation of tendon": ["tendonitis", "tendinitis", "inflammatory condition of a tendon"],
    "rheumatic fever": ["autoimmune inflammatory fever", "fever caused by streptococcal infection", "acute rheumatic fever"],
    "rheumatic arthritis": ["rheumatoid arthritis", "inflammatory joint condition", "autoimmune arthritis"],
    "bone fracture": ["broken bone", "bone break", "fractured bone"],
    "low bone density": ["osteopenia", "decreased bone mass", "reduced bone density"],
    "bone loss in the jaw": ["jawbone deterioration", "mandibular bone loss", "jawbone thinning"],
    "elevated lipids": ["high lipids", "high levels of fats in the blood", "hyperlipidemia"],
    "cholesterol": ["blood cholesterol", "high cholesterol levels", "cholesterol in the blood"],
    "triglycerides": ["blood triglycerides", "high triglyceride levels", "triglycerides in the blood"],
    "low-density lipoprotein": ["LDL cholesterol", "bad cholesterol", "low-density lipids"],
    "obesity": ["overweight", "excessive body weight", "obese"],
    "Rheumatoid arthritis": ["RA", "inflammatory arthritis", "autoimmune joint condition"],
    "treating headache": ["headache relief", "headache treatment", "alleviating headache"],
    "nerve related pains": ["neuralgia", "neuropathic pain", "pain from nerve disorders"],
    "fistula in ano": ["anal fistula", "perianal fistula", "abnormal tunnel between the anus and the skin"],
    "hernia": ["herniation", "protrusion of an organ through an abnormal opening"],
     "frozen shoulder headache": ["headache due to frozen shoulder", "shoulder pain headache", "headache from frozen shoulder"],
    "urinary bladder pain": ["pain in the bladder", "bladder discomfort", "bladder ache"],
    "early arthritis": ["arthritis in early stage", "preliminary arthritis", "initial arthritis"],
    "digestion troubles": ["digestive problems", "digestive issues", "trouble with digestion"],
    "swelling": ["inflammation", "edema", "puffiness"],
    "stiffness and inflammation associated with gout and rheumatoid arthritis": ["joint stiffness in gout and RA", "stiffness due to gout and rheumatoid arthritis", "joint rigidity from gout and RA"],
   "infection": ["infectious condition", "bacterial infection", "viral infection"],
    "suppuration": ["formation of pus", "pus formation", "purulence"],
    "discharge of pus": ["pus discharge", "pus drainage"],
    "sinuses": ["sinus cavities", "sinus passages", "nasal sinuses"],
    "skin infection": ["cutaneous infection", "dermal infection", "skin ailment"],
    "non-healing wounds": ["wounds that don't heal", "persistent wounds", "slow-healing wounds"],
    "cramps": ["muscle cramps", "spasms", "muscle contractions"],
    "stomach upset": ["upset stomach", "gastric discomfort", "stomach trouble"],
    "and diarrhea": ["with diarrhea", "accompanied by diarrhea"],
    "belching": ["burping", "eructation", "belch"],
   "indigestion and loss of appetite": ["digestive troubles and reduced appetite", "poor digestion and lack of hunger"],
    "alleviating hemiplegia": ["relieving hemiplegic symptoms", "improving hemiplegia"],
    "sciatica pain": ["pain associated with sciatica", "pain caused by sciatic nerve compression"],
    "sciatica": ["sciatic nerve pain", "sciatic nerve compression"],
    "lower backache": ["pain in the lower back", "lower back pain"],
    "joint disorders": ["joint ailments", "problems with joints", "joint issues"],
    "muscles": ["muscular system", "musculature", "muscle tissue"],
    "bone pain": ["pain in the bones", "skeletal pain"],
   "fibromyalgia": ["fibromyalgia syndrome", "fibromyalgia disorder"],
    "joint stiffness": ["stiffness in the joints", "restricted joint movement"],
    "irritable bowel syndrome": ["IBS", "intestinal sensitivity", "nervous stomach"],
    "sciatic nerve pain": ["sciatica pain", "pain due to sciatic nerve compression"],
    "premenstrual syndrome": ["PMS", "premenstrual tension", "premenstrual disorder"],
    "heaviness in the abdomen": ["abdominal heaviness", "feeling of weight in the abdomen"],
    "indigestion": ["dyspepsia", "upset stomach", "gastric discomfort"],
   "sticky stools": ["stool adhesion", "feces adhering to the rectum"],
    "metabolic deficiencies": ["metabolic insufficiencies", "metabolic imbalances"],
    "abdominal gas": ["stomach gas", "intestinal gas", "bloating"],
    "colic pain": ["colicky pain", "spasmodic abdominal pain"],
    "flatulence indigestion": ["flatulence with indigestion", "gas and indigestion"],
    "gastritis": ["stomach inflammation", "gastric irritation"],
    "loss of appetite": ["appetite loss", "decreased appetite"],
    "diabetes": ["diabetes mellitus", "high blood sugar"],
    "high cholesterol": ["hypercholesterolemia", "elevated cholesterol levels"],
    "heart diseases": ["cardiac diseases", "cardiovascular disorders"],
    "high BP": ["hypertension", "elevated blood pressure"],
    "diarrhea and indigestion": ["diarrheal symptoms with indigestion", "digestive upset with diarrhea"],
    "Psychosis": ["psychotic disorder", "severe mental disorder"],
    "Neurosis": ["neurotic disorder", "emotional disorder"],
    "Hysteria": ["hysterical disorder", "conversion disorder"],
    "Speech disorders": ["speech impediments", "speech abnormalities"],
    "Rhinorrhoea": ["runny nose", "nasal discharge"],
    "Erysipelas": ["skin infection", "cellulitis"],
    "Rheumatoid disease": ["rheumatoid disorder", "autoimmune joint disease"],
   "U.T.I.": ["urinary tract infection", "bladder infection"],
    "Male and Female subfertility": ["infertility in both males and females", "reproductive issues"],
    "Chronic toxicities": ["persistent toxic effects", "long-term exposure to toxins"],
    "Pruritus": ["itching", "skin irritation"],
    "Hyper Acidity": ["excessive stomach acid", "acidic stomach"],
    "Abdominal Burning pain": ["burning sensation in the abdomen", "stomach burning"],
    "Gastric Ulcers": ["stomach ulcers", "peptic ulcers"],
    "Duodinal Ulcers": ["ulcers in the duodenum", "duodenal ulcers"],
    "Ulcerative Colitis": ["inflammatory bowel disease", "colonic ulcers"],
   "mild psoriasis": ["psoriasis with mild symptoms", "mild skin disorder"],
    "Brain disorder": ["neurological disorder", "cerebral disorder"],
    "eyes and bone marrow nerves disorder": ["ocular and bone marrow nerve disorder"],
    "lower back pain": ["lumbar pain", "pain in the lower back"],
    "weakness of the uterus": ["uterine weakness", "weakened uterine muscles"],
    "infertility in women": ["female infertility", "difficulty conceiving in women"],
    "low semen secretion": ["reduced semen production", "decreased semen volume"],
     "poor conceiving": ["difficulty conceiving", "trouble with conception"],
    "renal colic": ["kidney stone pain", "renal stone colic"],
    "urinary tract infection": ["UTI", "bladder infection"],
    "Phthisis": ["tuberculosis", "lung disease"],
    "Hyperacidity": ["excessive acidity", "acid reflux"],
    "Urogenital disorders": ["issues with the urogenital system", "urological disorders"],
    "Vaginal diseases": ["conditions affecting the vagina", "vaginal disorders"],
    "Haemorrhoids": ["piles", "hemorrhoidal disease"],
    "Hydrocele": ["swelling in the scrotum", "fluid-filled sac around the testicles"],
    "Inflammatory Bowel Disease": ["IBD", "chronic bowel inflammation"],
   "White Discharge": ["leucorrhea", "vaginal discharge"],
    "Seizures": ["convulsions", "fits"],
    "eye pain with itching": ["ocular discomfort with itchiness", "itchy eye pain"],
    "weak immunity": ["low immune function", "impaired immune system"],
    "dryness of eye": ["ocular dryness", "dry eye syndrome"],
    "hairloss": ["hair loss", "alopecia"],
    "burning sensation in body": ["generalized body burning", "systemic burning sensation"],
    "Pain and swelling due to joint disorders": ["arthritic pain and swelling", "joint disorder-related discomfort and swelling"],
    "Backache": ["lower back pain", "spinal discomfort"],
   "bad breath": ["halitosis", "foul breath"],
    "swollen gums": ["gum inflammation", "gum swelling"],
    "pain while chewing": ["discomfort during chewing", "aching while chewing"],
    "loosening of teeth": ["tooth mobility", "teeth becoming loose"],
    "excessive salivation": ["drooling", "hypersalivation"],
    "chest pain": ["sternalgia", "thoracic discomfort"],
    "throat infection": ["pharyngeal infection", "sore throat"],
    "oral cavity infection": ["mouth infection", "infection in the oral cavity"],
    "bowel movement": ["defecation", "stool passage"],
    "Heavy Stomach": ["abdominal heaviness", "feeling of a heavy stomach"],
   "High blood sugar": ["hyperglycemia", "elevated blood glucose levels"],
    "Hyperglycemia": ["high blood glucose", "elevated blood sugar levels"],
    "Weight management": ["weight control", "managing body weight"],
    "painful periods": ["dysmenorrhea", "menstrual pain"],
    "mid-cycle bleeding": ["intermenstrual bleeding", "bleeding between periods"],
    "hypertension": ["high blood pressure", "elevated blood pressure"],
    "irregular heartbeat": ["arrhythmia", "abnormal heart rhythm"],
    "cholestrol": ["cholesterol", "lipid levels"],
    "shortness of breath": ["dyspnea", "breathlessness"],
    "cardiac weakness": ["heart weakness", "weak heart"],
    "inflammation in air passage": ["respiratory inflammation", "airway inflammation"],
    "nervousness": ["anxiety", "nervous tension", "edginess"],
    "low concentration": ["poor focus", "lack of concentration"],
    "weak memory": ["memory impairment", "poor memory"],
    "low energy": ["fatigue", "lack of energy", "tiredness"],
    "inflammation of the lining of the uterus": ["endometritis", "uterine lining inflammation"],
    "low stamina": ["lack of endurance", "poor physical stamina"],
    "wrinkles": ["fine lines", "facial creases"],
    "dark circles": ["under-eye circles", "periorbital darkening"],
    "urinary tract disorder": ["urinary system disorder", "urological condition"],
   "urinary calculi": ["urinary stones", "kidney stones"],
    "abdominal colic": ["abdominal spasms", "colicky pain"],
    "urinary obstruction": ["blocked urine flow", "urinary blockage"],
    "abdominal tumours": ["abdominal growths", "tumors in the abdomen"],
    "low volume of semen": ["reduced semen volume", "oligospermia"],
    "low sperm count": ["reduced sperm count", "oligospermia"],
    "heavy thirst": ["excessive thirst", "polydipsia"],
    "poor digestive system": ["impaired digestion", "digestive issues"],
    "Edema": ["swelling", "fluid retention"],
    "heart burn": ["heartburn", "acid reflux"],
    "drizziness.": ["dizziness", "vertigo"],
    "ascitis": ["ascites", "abdominal fluid retention"],
    "inflammation in reproductory organs": ["reproductive organ inflammation"],
    "poor concieving": ["difficulty conceiving", "trouble conceiving"],
    "Rectal pain": ["pain in rectum", "rectal discomfort"],
    "Brachial neuritis": ["neuralgic pain in the brachial plexus", "brachial plexus inflammation"],
    "Acidity": ["acidic stomach", "excessive acidity"],
    "Low Appetite": ["decreased appetite", "poor appetite"],
    "respiratory disorders": ["breathing issues", "lung problems"],
    "abdominal discomfort": ["uncomfortable abdomen", "abdominal unease"],
    "soft stools": ["loose stools", "diarrhea"],
    "Gastric disorders": ["stomach problems", "gastric issues"],
    "bleeding gum": ["bleeding gums", "gum bleeding"],
    "tooth ache": ["toothache", "dental pain"],
    "pyorrhoea": ["periodontal disease", "gum disease"],
   "pyorrhoea": ["periodontal disease", "gum disease"],

   # Add more synonyms
 }


# Function to map synonyms to recognized symptoms
def map_synonyms_to_symptoms(input_symptoms):
    mapped_symptoms = []
    for symptom in input_symptoms:
        for key, synonyms in synonym_dict.items():
            if symptom in synonyms:
                mapped_symptoms.append(key)
                break
        else:
            # If the symptom is not found in synonyms, use fuzzy matching to find the closest known symptom
            closest_match = max(synonym_dict.keys(), key=lambda known_symptom: fuzz.partial_ratio(symptom.lower(), known_symptom.lower()))
            if fuzz.partial_ratio(symptom.lower(), closest_match.lower()) > 80:  # You can adjust the similarity threshold as needed
                mapped_symptoms.append(closest_match)
    return mapped_symptoms


# Inputted symptoms (preprocessed and tokenized)
user_input = input("Enter symptoms: ")
user_symptoms = user_input.split(',')

user_symptoms_lower = [symptom.lower() for symptom in user_symptoms]
user_symptoms_mapped = map_synonyms_to_symptoms(user_symptoms_lower)
user_symptoms_stemmed = [stemmer.stem(symptom) for symptom in user_symptoms_mapped]
user_symptoms = ','.join(user_symptoms_stemmed)


def stem_words(word_list):
    return [stemmer.stem(word) for word in word_list]



def tokenize_symptoms(symptoms):
    # Split symptoms based on commas and treat them as individual tokens
    tokens = [token.strip() for token in symptoms.split(',') if token.strip()]
    return tokens

# Apply the symptom tokenization function to the combined_text


# Preprocess and tokenize the "Symptoms" column using the custom tokenizer
df['Symptoms1'] = df1['Symptoms'].apply(tokenize_symptoms)

# Tokenize the input symptoms
input_symptoms_tokens1 = tokenize_symptoms(user_symptoms)
#input_symptoms_tokens1=stem_words(input_symptoms_tokens1)
df['Symptoms1'] = df1['Symptoms1'].apply(stem_words)

# Create TF-IDF vectors for symptoms
tfidf_vectorizer = TfidfVectorizer()
symptoms_tfidf = tfidf_vectorizer.fit_transform(df['Symptoms1'].apply(lambda x: ' '.join(x)))

# Create a TF-IDF vector for the input symptoms
input_symptoms_tfidf = tfidf_vectorizer.transform([' '.join(input_symptoms_tokens1)])

# Calculate cosine similarity between input symptoms and each formulation's symptoms
cosine_similarities = cosine_similarity(symptoms_tfidf, input_symptoms_tfidf)

# Add the similarity scores to the DataFrame
df['Similarity_Score'] = cosine_similarities

# Find the formulation with the highest similarity score
recommended_row = df[df['Similarity_Score'] == df['Similarity_Score'].max()]
recommended_formulation = recommended_row['Formulation'].values[0]
similarity_score = recommended_row['Similarity_Score'].values[0]
#print(recommended_formulation)
#print(similarity_score)
##########################################################################################################
# Tokenize the input symptoms
input_symptoms_tokens = [stemmer.stem(symptom.lower()) for symptom in user_input.split(',') if symptom.strip()]

# Tokenize and stem the symptoms in the DataFrame
df['Symptoms'] = df['Symptoms'].apply(lambda x: [stemmer.stem(symptom.lower()) for symptom in x.split(',') if symptom.strip()])

# Train Word2Vec embeddings on your symptom data
model = gensim.models.Word2Vec(df['Symptoms'],vector_size=100, window=5, min_count=1, sg=0)  # Adjust parameters as needed

# Function to get the vector representation of a list of words
def get_word_vectors(word_list, model):
    vectors = [model.wv[word] for word in word_list if word in model.wv]
    if not vectors:
        # If no valid word vectors found, return a zero vector
        return [0.0] * model.vector_size
    return sum(vectors) / len(vectors)

# Create vectors for user input symptoms
user_input_vector = get_word_vectors(input_symptoms_tokens, model)

# Create vectors for symptoms in the DataFrame
df['SymptomVectors'] = df['Symptoms'].apply(lambda x: get_word_vectors(x, model))

# Calculate cosine similarity between input symptoms and each formulation's symptoms
df['Similarity Score'] = df['SymptomVectors'].apply(lambda x: cosine_similarity([user_input_vector], [x])[0][0])

###########################################################################################################
# Sort formulations by Similarity Score (descending)
df['Exact Match'] = df['Symptoms'].apply(lambda x: any(symptom in user_symptoms for symptom in x))
df = df.sort_values(by=['Similarity Score','Similarity_Score','Exact Match'], ascending=[False,False, False]).reset_index(drop=True)
x=df['Similarity Score'].sum()
y=df['Similarity_Score'].sum()



# Number of top recommendations to display
top_n = 30

# Display the top N recommended formulations if there are any recommendations
if x!=0 or y!=0:
   if len(df) > 0 :
      print(f"Top {top_n} Recommendations:")
      for i in range(min(top_n, len(df))):
        if df['Similarity_Score'][i]!=0 or df['Similarity Score'][i]!=0:
                formulation = df.iloc[i]
                print(f"{formulation['Formulation']} - Ingredients: {formulation['Ingredients']}, Properties: {formulation['Ayurvedic Properties']},Symptoms: {formulation['Symptoms']}")
else:
    print("No recommendations available.")

Enter symptoms: high blood cerculation
Top 30 Recommendations:
Guggulutiktaka Ghrita - Ingredients: Guggul, Neem, Vacha, Properties: nan,Symptoms: ['diabet', ' high cholesterol', ' heart diseases ', 'high bp', 'diarrhoea and indigest']
Nishamalaki Churna - Ingredients: turmeric , amla, Properties: nan,Symptoms: ['diabet', ' high blood sugar', ' hyperglycemia', ' liver health', ' skin disord', ' digestive disord', ' weight manag', ' inflamm']
Arjuna Churna - Ingredients: resh stem bark of Arjuna, Madhuka, Draksha, Dhataki, and Jaggery, Properties: nan,Symptoms: ['high blood pressur', ' chest pain', ' hypertens', ' irregular heartbeat', ' cholestrol', ' shortness of breath', ' cardiac weak', ' swell', ' edema', ' bronchiti', ' inflammation in air passag']
Arjunarishta/ Parthadyarishta - Ingredients: Arjuna Mrudvika Madhuka, Properties: nan,Symptoms: ['angina pectori', ' heart', ' high blood pressur', ' myocardial infarct', ' congestive heart failur', ' isochemic cardiomyopathi', ' cardia